# Data Preprocessing

author: Melih Kacaman
date: 05-10-2022

# Constants

In [6]:
import warnings
import sys 

warnings.filterwarnings("ignore")
sys.path.insert(0, "..") 

DATASET1_ODBC = 'PROTO.Melih.Dataset1_SimilarProducts'
DATASET1_TITLE = 'dataset1'

## Connect to the MSSQL Server

In [7]:
from common.sql_helper import custom_query
import pandas as pd


query = f'select * from {DATASET1_ODBC}'
result = custom_query(query=query)

result.head()

,WebSecenek,MarkaKodu,CinsiyetKodu,UrunGrubuKodu,Renk,cUrl,UrlNo
0,30006866-300,08,1,0GU,300,https://resim.aydinli.com.tr/08/1/0GU/S3000686...,2
1,30006866-850,08,1,0GU,850,https://resim.aydinli.com.tr/08/1/0GU/S3000686...,2
2,30006923-350,08,1,0GK,350,https://resim.aydinli.com.tr/08/1/0GK/S3000692...,2
3,30006925-350,08,1,0GK,350,https://resim.aydinli.com.tr/08/1/0GK/S3000692...,2
4,30006925-850,08,1,0GK,850,https://resim.aydinli.com.tr/08/1/0GK/S3000692...,2


# Download Images

In [8]:
class_names = result[['CinsiyetKodu', 'UrunGrubuKodu', 'Renk']].drop_duplicates()
class_names.head()

,CinsiyetKodu,UrunGrubuKodu,Renk
0,1,0GU,300
1,1,0GU,850
2,1,0GK,350
4,1,0GK,850
5,1,BGU,001


In [9]:
class_names.shape

(87, 3)

In [10]:
import requests

def save_image(image_url, path):
    img_data = requests.get(image_url).content
    with open(path, 'wb') as handler:
        handler.write(img_data)

In [11]:
import requests
from PIL import Image
import io

def save_image_resize(image_url, path):
    img_data = requests.get(image_url).content

    with Image.open(io.BytesIO(img_data)) as im:
        im = im.resize((224, 224), Image.ANTIALIAS)
        im.save(path, optimize=True, quality=95)

In [13]:
import numpy as np
import cv2
import requests

def save_image_cv(url, scale_percent, path):
    resp = requests.get(url, stream=True).raw
    image = np.asarray(bytearray(resp.read()), dtype="uint8")
    image = cv2.imdecode(image, cv2.IMREAD_COLOR)

    width = int(image.shape[1] * scale_percent / 100)
    height = int(image.shape[0] * scale_percent / 100)

    dsize = (width, height)
    output = cv2.resize(image, dsize)

    cv2.imwrite(path, output)

In [27]:
import os

def make_dataset(bran_id, dataset_name, datasource, iteration = None):
    try:
        base = os.path.abspath(os.path.join(os.path.dirname(os.getcwd()), '.'))
        path = os.path.join('datasets', dataset_name)
        path = os.path.join(base, path)
        if not os.path.isdir(path):
            os.mkdir(path)
        path = os.path.join(path, f'brand_{bran_id}')
        if not os.path.isdir(path):
            os.mkdir(path)
        # datasets/dataset/brand_id

        iterator = 0
        for index, row in class_names.iterrows():
            _class = row['CinsiyetKodu'] + '-'+ row['UrunGrubuKodu'] + '-' + row['Renk']
            folder = os.path.join(path, _class)
            res = datasource.query(f'MarkaKodu == "{bran_id}" and CinsiyetKodu == "{row.CinsiyetKodu}" and UrunGrubuKodu == "{row.UrunGrubuKodu}" and Renk == "{row.Renk}"')

            # if exists any data
            if res.shape[0] > 0:
                # check the folder exist
                if not os.path.isdir(folder):
                    os.mkdir(folder)

                # add the imgs to the folder
                for img_index, img_row in res.iterrows():
                    save_image_resize(img_row['cUrl'], os.path.join(folder, str(img_index) + '.jpg'))
                    if iteration is not None and iterator == iteration:
                        break
                    else:
                        iterator+=1
    except Exception as e:
        print('An exception occurred.', e)

In [14]:
import os

def make_dataset_resize(bran_id, dataset_name, datasource, iteration = None):
    problems_of_paths = []
    try:
        base = os.path.abspath(os.path.join(os.path.dirname(os.getcwd()), '.'))
        path = os.path.join('datasets', dataset_name)
        path = os.path.join(base, path)
        if not os.path.isdir(path):
            os.mkdir(path)
        path = os.path.join(path, f'brand_{bran_id}')
        if not os.path.isdir(path):
            os.mkdir(path)
        # datasets/dataset/brand_id

        iterator = 0
        for index, row in class_names.iterrows():
            _class = row['CinsiyetKodu'] + '-'+ row['UrunGrubuKodu'] + '-' + row['Renk']
            folder = os.path.join(path, _class)
            res = datasource.query(f'MarkaKodu == "{bran_id}" and CinsiyetKodu == "{row.CinsiyetKodu}" and UrunGrubuKodu == "{row.UrunGrubuKodu}" and Renk == "{row.Renk}"')

            # if exists any data
            if res.shape[0] > 0:
                # check the folder exist
                if not os.path.isdir(folder):
                    os.mkdir(folder)

                # add the imgs to the folder
                for img_index, img_row in res.iterrows():
                    try:
                        save_image_resize(img_row['cUrl'], os.path.join(folder, str(img_index) + '.jpg'))
                        if iteration is not None and iterator == iteration:
                            break
                        else:
                            iterator+=1
                    except Exception as e2:
                        problems_of_paths.append(os.path.join(folder, str(img_index) + '.jpg'))
                        continue

    except Exception as e:
        print('An exception occurred.', e)

    return problems_of_paths

In [18]:
import os

def make_dataset_cv(bran_id, dataset_name, datasource, iteration = None):
    problems_of_paths = []
    try:
        base = os.path.abspath(os.path.join(os.path.dirname(os.getcwd()), '.'))
        path = os.path.join('datasets', dataset_name)
        path = os.path.join(base, path)
        if not os.path.isdir(path):
            os.mkdir(path)
        path = os.path.join(path, f'brand_{bran_id}')
        if not os.path.isdir(path):
            os.mkdir(path)
        # datasets/dataset/brand_id

        folder = os.path.join(path, "all") 
        os.mkdir(folder)
        
        for img_index, img_row in datasource.iterrows():
                try:
                    save_image_cv(img_row['cUrl'], 50, os.path.join(folder, str(img_index) + '.png'))
                    if iteration is not None and iterator == iteration:
                        break
                    else:
                        iterator+=1
                except Exception as e2:
                    problems_of_paths.append(os.path.join(folder, str(img_index) + '.jpg'))
                    print(e2)
                    continue
    except Exception as e:
        print('An exception occurred.', e)
    return problems_of_paths

In [20]:
%%time
errors = make_dataset_cv('08', DATASET1_TITLE, result)
errors

'NoneType' object has no attribute 'shape'
local variable 'iterator' referenced before assignment
'NoneType' object has no attribute 'shape'
'NoneType' object has no attribute 'shape'
'NoneType' object has no attribute 'shape'
local variable 'iterator' referenced before assignment
local variable 'iterator' referenced before assignment
local variable 'iterator' referenced before assignment
'NoneType' object has no attribute 'shape'
local variable 'iterator' referenced before assignment
local variable 'iterator' referenced before assignment
local variable 'iterator' referenced before assignment
local variable 'iterator' referenced before assignment
local variable 'iterator' referenced before assignment
local variable 'iterator' referenced before assignment
local variable 'iterator' referenced before assignment
local variable 'iterator' referenced before assignment
local variable 'iterator' referenced before assignment
local variable 'iterator' referenced before assignment
local variable 

['c:\\Users\\melih.kacaman\\projects_analytics\\similar-products\\SimilarProducts\\SimilarProducts\\datasets\\dataset1\\brand_08\\all\\0.jpg',
 'c:\\Users\\melih.kacaman\\projects_analytics\\similar-products\\SimilarProducts\\SimilarProducts\\datasets\\dataset1\\brand_08\\all\\1.jpg',
 'c:\\Users\\melih.kacaman\\projects_analytics\\similar-products\\SimilarProducts\\SimilarProducts\\datasets\\dataset1\\brand_08\\all\\2.jpg',
 'c:\\Users\\melih.kacaman\\projects_analytics\\similar-products\\SimilarProducts\\SimilarProducts\\datasets\\dataset1\\brand_08\\all\\3.jpg',
 'c:\\Users\\melih.kacaman\\projects_analytics\\similar-products\\SimilarProducts\\SimilarProducts\\datasets\\dataset1\\brand_08\\all\\4.jpg',
 'c:\\Users\\melih.kacaman\\projects_analytics\\similar-products\\SimilarProducts\\SimilarProducts\\datasets\\dataset1\\brand_08\\all\\5.jpg',
 'c:\\Users\\melih.kacaman\\projects_analytics\\similar-products\\SimilarProducts\\SimilarProducts\\datasets\\dataset1\\brand_08\\all\\6.jpg',